In [2]:
%pip install -q transformers peft ipywidgets torch

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import ipywidgets as widgets
from IPython.display import display, Markdown

device = "mps" if torch.backends.mps.is_available() else "cpu"
MODEL_NAME = "google/flan-t5-base"

print(device)


Note: you may need to restart the kernel to use updated packages.
mps


In [3]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
model = PeftModel.from_pretrained(base, "../artifacts/t5_lora_best").to(device)
print("Modelo fine-tunado carregado")


Modelo fine-tunado carregado


In [4]:
def generate_description(title, beams=2, temp=0.9, top_p=0.9, rep_penalty=1.8, no_repeat=3):
    prompt = f"Given a product title, generate its product description.\nTitle: {title}\nDescription:"
    enc = tok(prompt, return_tensors="pt", truncation=True, max_length=128).to(device)
    out = model.generate(
        **enc,
        max_new_tokens=224,
        num_beams=beams,
        temperature=temp,
        top_p=top_p,
        repetition_penalty=rep_penalty,
        no_repeat_ngram_size=no_repeat,
        do_sample=True,
    )
    return tok.decode(out[0], skip_special_tokens=True)

In [6]:
title_input = widgets.Text(
    value="GoFit Weightlifting Glove",
    description='Title:',
    layout=widgets.Layout(width="600px")
)
beams_slider = widgets.IntSlider(value=2, min=1, max=5, description='Feixes (num_beams)')
temp_slider = widgets.FloatSlider(value=0.9, min=0.5, max=1.5, step=0.05, description='Temperatura')
top_p_slider = widgets.FloatSlider(value=0.9, min=0.5, max=1.0, step=0.05, description='Top P')
rep_slider = widgets.FloatSlider(value=1.8, min=1.0, max=3.0, step=0.1, description='Penalidade de repetição')
norep_slider = widgets.IntSlider(value=3, min=1, max=5, description='Sem repetição de n-gramas')

button = widgets.Button(description="Gerar descrição", button_style='success')


out_box = widgets.Output()

def on_click(_):
    with out_box:
        out_box.clear_output()
        desc = generate_description(
            title_input.value,
            beams_slider.value,
            temp_slider.value,
            top_p_slider.value,
            rep_slider.value,
            norep_slider.value
        )
        display(Markdown(f"### 🧾 Description\n{desc}"))

button.on_click(on_click)

display(title_input, beams_slider, temp_slider, top_p_slider, rep_slider, norep_slider, button, out_box)

Text(value='GoFit Weightlifting Glove', description='Title:', layout=Layout(width='600px'))

IntSlider(value=2, description='Feixes (num_beams)', max=5, min=1)

FloatSlider(value=0.9, description='Temperatura', max=1.5, min=0.5, step=0.05)

FloatSlider(value=0.9, description='Top P', max=1.0, min=0.5, step=0.05)

FloatSlider(value=1.8, description='Penalidade de repetição', max=3.0, min=1.0)

IntSlider(value=3, description='Sem repetição de n-gramas', max=5, min=1)

Button(button_style='success', description='Gerar descrição', style=ButtonStyle())

Output()